In [ ]:
!pip uninstall kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!ls -lha kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

Uninstalling kaggle-1.5.10:
  Would remove:
    /usr/local/bin/kaggle
    /usr/local/lib/python3.7/dist-packages/kaggle-1.5.10.dist-info/*
    /usr/local/lib/python3.7/dist-packages/kaggle/*
Proceed (y/n)? y
  Successfully uninstalled kaggle-1.5.10
     |████████████████████████████████| 1.5MB 8.7MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 3.4 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=8b1c75b1a6ac848b44a7a2f285b015014ca3ed540c91c8f0c5e01e7142da6abb
  Stored in directory: /root/.cache/pip/wheels/aa/e7/e7/eb3c3d514c33294d77ddd5a856bdd58dc9c1fabbed59a02a2b
Successfully built kaggle
-rw-r--r-- 1 root root 65 Mar 16 05:16 kaggle.json


In [ ]:
!kaggle competitions download -c tabular-playground-series-mar-2021

 75% 41.0M/54.9M [00:01<00:00, 26.7MB/s]
100% 54.9M/54.9M [00:01<00:00, 55.3MB/s]


In [ ]:
!unzip tabular-playground-series-mar-2021.zip

Archive:  tabular-playground-series-mar-2021.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
!pip install optuna


     |████████████████████████████████| 293 kB 8.8 MB/s 
     |████████████████████████████████| 80 kB 5.0 MB/s 
     |████████████████████████████████| 159 kB 13.5 MB/s 
     |████████████████████████████████| 75 kB 3.8 MB/s 
     |████████████████████████████████| 49 kB 5.1 MB/s 
     |████████████████████████████████| 133 kB 13.0 MB/s 
     |████████████████████████████████| 106 kB 14.1 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11107 sha256=42d88615321ae4b60eeb5f5d6b67a4c2df89608fc7c155cd841a95091805bfb0
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import numpy as np
import pandas as pd

from lightgbm import LGBMClassifier

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances

from IPython.display import display

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

train.head(10)

# cat0-cat18 : categorical
# cont0-cont10 : continuous

In [ ]:
predictors = train.columns[:-1]
target = train.columns[-1]


In [ ]:
# DATA PROCESSING


# 이 과정은 왜 ? ? ?
cat_cols = [col for col in predictors if 'cat' in col]

train[cat_cols] = train[cat_cols].astype('category')
train[target] = train[target].astype('category')

test[cat_cols] = test[cat_cols].astype('category')

In [ ]:
# Functions for KFold evaluation
def create(hyperparams):
    """Create LGBM Classifier for a given set of hyper-parameters."""
    model = LGBMClassifier(**hyperparams)
    return model

def fit_with_stop(model, X, y, X_val, y_val):
    """Advanced training with early stopping."""
    model.fit(X, y,
              eval_set=(X_val, y_val),
              early_stopping_rounds=200, # ! Hard-coded value
              verbose=100)
    return model

def evaluate(model, X, y):
    """Compute AUC for a given model."""
    yp = model.predict_proba(X)[:, 1]
    auc_score = roc_auc_score(y, yp)
    return auc_score

def kfold_evaluation(X, y, k, hyperparams):
    """Run a KFlod evaluation."""
    scores = []
    
    print(f"\n------ {k}-fold evaluation -----")
    print(hyperparams)
    
    kf = KFold(k)
    for i, (train_idx, test_idx) in enumerate(kf.split(X)):
        print(f"----- FOLD {i} -----")
        
        X_train = X.iloc[train_idx]
        y_train = y.iloc[train_idx]
        X_val = X.iloc[test_idx]
        y_val = y.iloc[test_idx]
        
        model = create(hyperparams) #하이퍼파라미터 넣어서 모델 생성 lgbm
        model = fit_with_stop(model, X_train, y_train, X_val, y_val) # 생성한 모델로 early_stop 사용해서 학습시키기
        train_score = evaluate(model, X_train, y_train) # 학습한 모델로 train set 정확도 측정 (roc_auc_score 사용)
        val_score = evaluate(model, X_val, y_val) # 학습한 모델로 validation set 정확도
        scores.append((train_score, val_score)) # 괄호 두 개? 데이터 프레임 형태로 넣어주는 거?
        
        print(f"Eval AUC: {val_score}")
        
        
    scores = pd.DataFrame(scores, columns=['train score', 'validation score'])
    
    return scores

def kfold_prediction(X, y, X_test, k, hyperparams):
    """Make predictions with a bagged model based on KFold."""
    yp = np.zeros(len(X_test))
    
    kf = KFold(k)
    for train_idx, test_idx in kf.split(X):
        X_train = X.iloc[train_idx]
        y_train = y.iloc[train_idx]
        X_val = X.iloc[test_idx]
        y_val = y.iloc[test_idx]
        
        model = create(hyperparams)
        model = fit_with_stop(model, X_train, y_train, X_val, y_val)
        yp += model.predict_proba(X_test)[:, 1] / k
    
    return yp

hyperparameter optimize 방법

https://www.freecodecamp.org/news/hyperparameter-optimization-techniques-machine-learning/

In [ ]:
# constant

k = 5
x = train[predictors]
y = train[target]
x_test = test[predictors]
best_params = {'n_estimators': 10000, 'learning_rate': 0.03, 'metric' : 'auc'}

In [ ]:
# Objective function
def objective(trial):
    # Search spaces
    hyperparams = {
        'num_leaves': trial.suggest_int('num_leaves', 2, 1024),
        'max_depth': trial.suggest_int('max_depth', 1, 32),
        'reg_alpha': trial.suggest_float('reg_alpha', 1E-16, 25),
        'reg_lambda': trial.suggest_float('reg_lambda', 1E-16, 25),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 1E-16, 1.0),
        'subsample': trial.suggest_float('subsample ', 1E-16, 1.0),
        'cat_smooth': trial.suggest_float('cat_smooth', 1.0, 50.0)  
    }
    
    # Add BEST_PARAMS
    hyperparams.update(best_params)
    
    # Evaluation
    scores = kfold_evaluation(x, y, k, hyperparams)
    
    return scores['validation score'].mean()


In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, timeout=3600*8)

[I 2021-03-16 05:17:31,508] A new study created in memory with name: no-name-2c402abd-858d-4f33-9d92-cee34f76e7fd



------ 5-fold evaluation -----
{'num_leaves': 516, 'max_depth': 23, 'reg_alpha': 19.395942528101635, 'reg_lambda': 12.091327802510696, 'colsample_bytree': 0.8770305749389554, 'subsample': 0.7120266356404445, 'cat_smooth': 41.58882940921926, 'n_estimators': 10000, 'learning_rate': 0.03, 'metric': 'auc'}
----- FOLD 0 -----
Training until validation scores don't improve for 200 rounds.
[100]	valid_0's auc: 0.889287
[200]	valid_0's auc: 0.892942
[300]	valid_0's auc: 0.894159
[400]	valid_0's auc: 0.89469
[500]	valid_0's auc: 0.894826
[600]	valid_0's auc: 0.894776
[700]	valid_0's auc: 0.894685
Early stopping, best iteration is:
[514]	valid_0's auc: 0.894862
Eval AUC: 0.894862351269181
----- FOLD 1 -----
Training until validation scores don't improve for 200 rounds.
[100]	valid_0's auc: 0.888519
[200]	valid_0's auc: 0.893026
[300]	valid_0's auc: 0.894536
[400]	valid_0's auc: 0.895069
[500]	valid_0's auc: 0.895246
[600]	valid_0's auc: 0.895168
[700]	valid_0's auc: 0.894965
Early stopping, bes

[I 2021-03-16 05:28:17,370] Trial 0 finished with value: 0.8941873297270474 and parameters: {'num_leaves': 516, 'max_depth': 23, 'reg_alpha': 19.395942528101635, 'reg_lambda': 12.091327802510696, 'colsample_bytree': 0.8770305749389554, 'subsample ': 0.7120266356404445, 'cat_smooth': 41.58882940921926}. Best is trial 0 with value: 0.8941873297270474.


Eval AUC: 0.893491313964029

------ 5-fold evaluation -----
{'num_leaves': 196, 'max_depth': 17, 'reg_alpha': 3.2395910914905013, 'reg_lambda': 3.885042508165182, 'colsample_bytree': 0.5889516804996797, 'subsample': 0.06007884501479608, 'cat_smooth': 16.950701055943924, 'n_estimators': 10000, 'learning_rate': 0.03, 'metric': 'auc'}
----- FOLD 0 -----
Training until validation scores don't improve for 200 rounds.
[100]	valid_0's auc: 0.892366
[200]	valid_0's auc: 0.894882
[300]	valid_0's auc: 0.895928
[400]	valid_0's auc: 0.89631
[500]	valid_0's auc: 0.896325
[600]	valid_0's auc: 0.896122
Early stopping, best iteration is:
[440]	valid_0's auc: 0.896423
Eval AUC: 0.896422753286459
----- FOLD 1 -----
Training until validation scores don't improve for 200 rounds.
[100]	valid_0's auc: 0.891145
[200]	valid_0's auc: 0.894404
[300]	valid_0's auc: 0.895756
[400]	valid_0's auc: 0.896333
[500]	valid_0's auc: 0.896491
[600]	valid_0's auc: 0.896458
Early stopping, best iteration is:
[493]	valid_0's

[I 2021-03-16 05:34:47,464] Trial 1 finished with value: 0.8955967506180758 and parameters: {'num_leaves': 196, 'max_depth': 17, 'reg_alpha': 3.2395910914905013, 'reg_lambda': 3.885042508165182, 'colsample_bytree': 0.5889516804996797, 'subsample ': 0.06007884501479608, 'cat_smooth': 16.950701055943924}. Best is trial 1 with value: 0.8955967506180758.


...

Eval AUC: 0.8960646213918358

------ 5-fold evaluation -----
{'num_leaves': 718, 'max_depth': 10, 'reg_alpha': 10.496995448847034, 'reg_lambda': 8.361050682890738, 'colsample_bytree': 0.133061966567021, 'subsample': 0.31528530711733815, 'cat_smooth': 10.666989250916894, 'n_estimators': 10000, 'learning_rate': 0.03, 'metric': 'auc'}
----- FOLD 0 -----
Training until validation scores don't improve for 200 rounds.
[100]	valid_0's auc: 0.886528
[200]	valid_0's auc: 0.891287
[300]	valid_0's auc: 0.893579
[400]	valid_0's auc: 0.894694
[500]	valid_0's auc: 0.895406
[600]	valid_0's auc: 0.895887
[700]	valid_0's auc: 0.896298
[800]	valid_0's auc: 0.896536
[900]	valid_0's auc: 0.8967
[1000]	valid_0's auc: 0.896866
[1100]	valid_0's auc: 0.89701
[1200]	valid_0's auc: 0.897082
[1300]	valid_0's auc: 0.897181
[1400]	valid_0's auc: 0.897229
[1500]	valid_0's auc: 0.897242
[1600]	valid_0's auc: 0.897283
[1700]	valid_0's auc: 0.897286
Early stopping, best iteration is:
[1559]	valid_0's auc: 0.897305
Eva

[I 2021-03-16 09:44:21,893] Trial 22 finished with value: 0.8966291498578183 and parameters: {'num_leaves': 718, 'max_depth': 10, 'reg_alpha': 10.496995448847034, 'reg_lambda': 8.361050682890738, 'colsample_bytree': 0.133061966567021, 'subsample ': 0.31528530711733815, 'cat_smooth': 10.666989250916894}. Best is trial 20 with value: 0.8966588892637161.


Eval AUC: 0.8963096720768366

------ 5-fold evaluation -----
{'num_leaves': 659, 'max_depth': 9, 'reg_alpha': 16.33096051375575, 'reg_lambda': 8.349172213191544, 'colsample_bytree': 0.09731376802504435, 'subsample': 0.290769796894117, 'cat_smooth': 10.582017481397648, 'n_estimators': 10000, 'learning_rate': 0.03, 'metric': 'auc'}
----- FOLD 0 -----
Training until validation scores don't improve for 200 rounds.
[100]	valid_0's auc: 0.88324
[200]	valid_0's auc: 0.888163
[300]	valid_0's auc: 0.890783
[400]	valid_0's auc: 0.892392
[500]	valid_0's auc: 0.893252
[600]	valid_0's auc: 0.893872
[700]	valid_0's auc: 0.89435
[800]	valid_0's auc: 0.894726
[900]	valid_0's auc: 0.895036
[1000]	valid_0's auc: 0.895278
[1100]	valid_0's auc: 0.895472
[1200]	valid_0's auc: 0.895637
[1300]	valid_0's auc: 0.895811
[1400]	valid_0's auc: 0.895947
[1500]	valid_0's auc: 0.896092
[1600]	valid_0's auc: 0.896211
[1700]	valid_0's auc: 0.896334
[1800]	valid_0's auc: 0.896453
[1900]	valid_0's auc: 0.896497
[2000]	v

[I 2021-03-16 10:03:48,362] Trial 23 finished with value: 0.8960912996808788 and parameters: {'num_leaves': 659, 'max_depth': 9, 'reg_alpha': 16.33096051375575, 'reg_lambda': 8.349172213191544, 'colsample_bytree': 0.09731376802504435, 'subsample ': 0.290769796894117, 'cat_smooth': 10.582017481397648}. Best is trial 20 with value: 0.8966588892637161.


Eval AUC: 0.8958173189113137

------ 5-fold evaluation -----
{'num_leaves': 997, 'max_depth': 15, 'reg_alpha': 7.91037376515425, 'reg_lambda': 16.266466414712042, 'colsample_bytree': 0.09459061109965355, 'subsample': 0.31141919060443257, 'cat_smooth': 11.863388790347601, 'n_estimators': 10000, 'learning_rate': 0.03, 'metric': 'auc'}
----- FOLD 0 -----
Training until validation scores don't improve for 200 rounds.
[100]	valid_0's auc: 0.878226
[200]	valid_0's auc: 0.884262
[300]	valid_0's auc: 0.887504
[400]	valid_0's auc: 0.889525
[500]	valid_0's auc: 0.890817
[600]	valid_0's auc: 0.891765
[700]	valid_0's auc: 0.892349
[800]	valid_0's auc: 0.892833
[900]	valid_0's auc: 0.893208
[1000]	valid_0's auc: 0.893474
[1100]	valid_0's auc: 0.893683
[1200]	valid_0's auc: 0.893877
[1300]	valid_0's auc: 0.894062
[1400]	valid_0's auc: 0.8942
[1500]	valid_0's auc: 0.894345
[1600]	valid_0's auc: 0.894447
[1700]	valid_0's auc: 0.894533
[1800]	valid_0's auc: 0.894606
[1900]	valid_0's auc: 0.89465
[2000]

In [ ]:
plot_optimization_history(study)

NameError: ignored

In [ ]:
plot_param_importances(study)